In [51]:
from gym import Env
from gym.spaces import Discrete, Box, Tuple
from random import random
import numpy as np
from create_dataset import CreateDataset

In [52]:
# Building a custom environment
class FootballEnv(Env):
    def __init__(self):
        self.dataset = CreateDataset()

        self.sequence_length = 5
        self.steps_left = self.sequence_length

        # The amount of actions that a player can take
        self.action_size = len(self.dataset.good_events)

        # The currect action being taken
        self.state = None

        # The possible action space ( A discrete action )
        self.action_space = Discrete(self.action_size)

        # The observation ( The last 5 actions )
        self.observation_space = np.array(
            Tuple((
                Discrete(self.action_size),
                Discrete(self.action_size),
                Discrete(self.action_size),
                Discrete(self.action_size),
                Discrete(self.action_size))
            )
        )
        # self.observation_space = Box(low=np.array([0,0]), high=np.array([1,1]))

    def step(self, action):
        self.steps_left -= 1
        reward = random()
        

        self.state = np.zeros((self.action_size), dtype=np.float32)
        self.state[action] = 1
        info = {}
        return self.state, reward, self.steps_left == 0, info

    def render(self):
        

        pass

    def reset(self):
        self.sequence_length = 5
        self.steps_left = self.sequence_length
        self.state = np.zeros((self.action_size), dtype=np.float32)
        return self.state

In [54]:
# env = FootballEnv()
# print(env.observation_space.shape)
# env.step(1)
# env.reset()

INIT!
(5,)
STEP!
4
RESET!


array([0., 0., 0., 0., 0.], dtype=float32)